In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from pickle import load, dump

from scipy import stats
from sklearn.metrics import *
from sklearn.model_selection import KFold
from validate import validate
import matplotlib.pyplot as plt

def ld(f): return load(open(f, 'rb'))  # _pickle.load
def dp(what, fP): dump(what, open(fP, 'wb'))  # _pickle.dump

## Station-specific probabilistic regression

In [ ]:
n_runs = 20
data_dir = './data'
locations = ['muOsna', 'wernig', 'braunl', 'redlen']
station = locations[2]
print(station)

### Training phase

In [ ]:
## Changes according to station. Obtained from arch_search.ipynb
archs = {'muOsna': {'depth': 6, 'units': 64},
         'wernig': {'depth': 8, 'units': 64},
         'braunl': {'depth': 8, 'units': 32},
         'redlen': {'depth': 6, 'units': 64},
         'general': {'depth': 8, 'units': 64}}

best_depth = archs[station]['depth']
best_units = archs[station]['units']

In [ ]:
## Load the dataset
id_predictors   = np.loadtxt(data_dir+'/id_predictors/'+station+'.txt', dtype=np.int32, delimiter=',') - 1 #Matlab index start in 1

trn_x = np.load(data_dir + '/' + station + '/trn_x.npy')
trn_x = trn_x[:, id_predictors]
trn_y = np.load(data_dir + '/' + station + '/trn_y.npy')

## Use only days with rain
trn_x = trn_x[trn_y > 0]
trn_y = trn_y[trn_y > 0]
trn_y = np.log(trn_y) ## Log transform for normal output
print('Train dataset:', trn_x.shape, '---> ', trn_y.shape) 

In [ ]:
for r in range(n_runs):
    
    print('Run', r+1, '/', n_runs)
    
    cross_validation_models = []
    cross_validation_losses = []
    cross_validation_histos = []


    kf = KFold(n_splits=10, shuffle=True)
    for train_index, val_index in kf.split(trn_x, trn_y):

        batch_size = 64
        epochs = 256

        optimizer = tf.optimizers.Adam(learning_rate=0.001)
        negloglik = lambda y, dist: -dist.log_prob(y)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, restore_best_weights=True)

        activation = 'relu'

        model = tf.keras.Sequential()

        model.add(tf.keras.layers.InputLayer(input_shape=(trn_x.shape[1])))

        for d in range(best_depth):
            model.add(tf.keras.layers.Dense(best_units, activation=activation))

        
        ## Probabilistic output
        model.add(tf.keras.layers.Dense(2))
        model.add(tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(t[..., :1], 1e-3 + tf.math.softplus(0.05 * t[...,1:]))))
        model.compile(optimizer, negloglik)

        hist = model.fit(trn_x[train_index], trn_y[train_index], batch_size, epochs, validation_data=(trn_x[val_index], trn_y[val_index]), shuffle=True, verbose=0, callbacks=[early_stop])
        loss = model.evaluate(trn_x[val_index], trn_y[val_index], verbose=0)

        cross_validation_models.append(model)
        cross_validation_losses.append(loss)
        cross_validation_histos.append(hist)


    ## Pick the best model from cross validation
    idx_best = np.argmin(cross_validation_losses)

    best_model = cross_validation_models[idx_best]
    best_model.save('results/probabilistic_regression/specific/' + station + '/model' + str(r).zfill(2) + '.h5')
    best_hist = cross_validation_histos[idx_best]

    fig, ax = plt.subplots()
    ax.plot(best_hist.history['loss'], label='loss')
    ax.plot(best_hist.history['val_loss'], label='val_loss')
    fig.legend()
    
    fig.savefig('results/probabilistic_regression/specific/' + station + '/loss' + str(r).zfill(2) + '.png')

### Testing phase

In [ ]:
tst_t = np.load(data_dir + '/' + station + '/tst_t.npy')
tst_x = np.load(data_dir + '/' + station + '/tst_x.npy')
tst_x = tst_x[:, id_predictors]
tst_y = np.load(data_dir + '/' + station + '/tst_y.npy')

tst_t = tst_t[tst_y > 0]
tst_x = tst_x[tst_y > 0]
tst_y = tst_y[tst_y > 0]
print('Test dataset:', tst_x.shape, '--->', tst_y.shape)

In [ ]:
results = []

for r in range(n_runs):

    model = tf.keras.models.load_model('results/probabilistic_regression/specific/' + station + '/model' + str(r).zfill(2) + '.h5', compile=False)

    pred = model(tst_x)
    pred = pred.quantile(.5)
    pred = np.array([x.numpy() for x in pred])
    pred = np.exp(pred) ## Reverse back log transform 
    pred = np.squeeze(pred)
    results.append(pred)

results = np.array(results)
print('Results [runs x ensembles x predictions] =', results.shape)    

In [ ]:
leps_cosmo = validate([], tst_t, [0], [4], True, station)['LEPS'][9]
results_skill = []

for r in range(n_runs):
    leps       = validate(results[r], tst_t, [0], [4], False, station)['LEPS']
    skill = 1 - (leps / leps_cosmo)
    results_skill.append(skill)

print(station, 'Results:', 'min =', np.min(results_skill), 'median = ', np.median(results_skill), 'max =', np.max(results_skill))
np.save('results/probabilistic_regression/specific/' + station + '/results_skill.npy', results_skill)

## General probabilistic regression

In [ ]:
best_units = archs['general']['units']
best_depth = archs['general']['depth']
print(best_units, best_depth)

### Training phase

In [ ]:
trn_x = []
trn_y = []

for station in locations:
    
    trn_x.append(np.load(data_dir + '/' + station + '/trn_x.npy'))
    trn_y.append(np.load(data_dir + '/' + station + '/trn_y.npy'))

trn_x = np.concatenate(trn_x)
trn_y = np.concatenate(trn_y)

## Use only days with rain
trn_x = trn_x[trn_y > 0]
trn_y = trn_y[trn_y > 0]
trn_y = np.log(trn_y) ## Log transform for normal output
print('Training dataset:', trn_x.shape, '---> ', trn_y.shape)

In [ ]:
for r in range(n_runs):
    
    print('Run', r+1, '/', n_runs)
    
    cross_validation_models = []
    cross_validation_losses = []
    cross_validation_histos = []


    kf = KFold(n_splits=10, shuffle=True)
    for train_index, val_index in kf.split(trn_x, trn_y):

        batch_size = 64
        epochs = 256

        optimizer = tf.optimizers.Adam(learning_rate=0.001)
        negloglik = lambda y, dist: -dist.log_prob(y)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, restore_best_weights=True)

        activation = 'relu'

        model = tf.keras.Sequential()

        model.add(tf.keras.layers.InputLayer(input_shape=(trn_x.shape[1])))

        for d in range(best_depth):
            model.add(tf.keras.layers.Dense(best_units, activation=activation))

        
        ## Probabilistic output
        model.add(tf.keras.layers.Dense(2))
        model.add(tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(t[..., :1], 1e-3 + tf.math.softplus(0.05 * t[...,1:]))))
        model.compile(optimizer, negloglik)

        hist = model.fit(trn_x[train_index], trn_y[train_index], batch_size, epochs, validation_data=(trn_x[val_index], trn_y[val_index]), shuffle=True, verbose=0, callbacks=[early_stop])
        loss = model.evaluate(trn_x[val_index], trn_y[val_index], verbose=0)

        cross_validation_models.append(model)
        cross_validation_losses.append(loss)
        cross_validation_histos.append(hist)


    ## Pick the best model from cross validation
    idx_best = np.argmin(cross_validation_losses)

    best_model = cross_validation_models[idx_best]
    best_model.save('results/probabilistic_regression/general/model' + str(r).zfill(2) + '.h5')
    best_hist = cross_validation_histos[idx_best]

    fig, ax = plt.subplots()
    ax.plot(best_hist.history['loss'], label='loss')
    ax.plot(best_hist.history['val_loss'], label='val_loss')
    fig.legend()
    
    fig.savefig('results/probabilistic_regression/general/loss' + str(r).zfill(2) + '.png')

### Test phase

In [ ]:
for station in locations:
    
    tst_t = np.load(data_dir + '/' + station + '/tst_t.npy')
    tst_x = np.load(data_dir + '/' + station + '/tst_x.npy')
    tst_y = np.load(data_dir + '/' + station + '/tst_y.npy')
    tst_t = tst_t[tst_y > 0]
    tst_x = tst_x[tst_y > 0]
    tst_y = tst_y[tst_y > 0]
    print(tst_x.shape, '--->', tst_y.shape)
    
    results = []

    for r in range(n_runs):
        model = tf.keras.models.load_model('results/probabilistic_regression/general/model' + str(r).zfill(2) + '.h5', compile=False)
        pred = model(tst_x)
        pred = pred.quantile(.5)
        pred = np.squeeze(pred)
        pred = np.exp(pred) ## Reverse back log transform 
        results.append(pred)

    results = np.array(results)
    print('Results [runs x ensembles x predictions] =', results.shape)
    
    leps_cosmo = validate([], tst_t, [0], [4], True, station)['LEPS'][9]
    results_skill = []

    for r in range(n_runs):
        print(r)
        leps       = validate(results[r], tst_t, [0], [4], False, station)['LEPS']
        skill = 1 - (leps / leps_cosmo)
        results_skill.append(skill)

    print(station, 'Results:', 'min =', np.min(results_skill), 'median = ', np.median(results_skill), 'max =', np.max(results_skill))
    np.save('results/probabilistic_regression/general/' + station + '_results_skill.npy', results_skill)